# Optimizacion de hiperparametros

Setup

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [2]:
# !gsutil cp /home/clas_giulia_s/buckets/b1/datasets/competencia_02_fe_v01_undersampled.parquet /home/clas_giulia_s/datasets/

In [3]:
# base_path = '/home/clas_giulia_s/buckets/b1/'
base_path = '/Users/ignacio/MAESTRIA/DMEF/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_fe_modelito_undersampled.csv'

ganancia_acierto = 273000
costo_estimulo = 7000

semillas = [261823, 289871, 379817, 481301, 959723]

# data = pd.read_parquet(dataset_path + dataset_file)
data = pd.read_csv(dataset_path + dataset_file)

In [4]:
# meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
#                202001, 202002, 202003, 202004, 202005, 202006,
#                202007, 202008, 202009, 202010, 202011, 202012,
#                202101, 202102, 202103, 202104, 202105] # dejo afuera 202106 para test

meses_train = [202101, 202102, 202103] # dejo afuera 202104 para test

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(51606, 679)

In [5]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [6]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria']
w_train = data['clase_peso']

In [7]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [8]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 200),
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.5), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1,
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria, # eligir la clase
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    # print(f"Learning Rate: {params['learning_rate']}, Type: {type(params['learning_rate'])}")
    
    # Use callbacks for early stopping
    rounds = int(50 + 5/learning_rate[0])
    early_stopping_cb = lgb.early_stopping(stopping_rounds= rounds)
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=10000, # modificar, subit y subir... 
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [9]:
storage_name = "sqlite:///" + db_path + "optimizacion_lgbm_modelito.db"
study_name = "competencia1_modelito_lgbm" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-29 00:11:15,865] A new study created in RDB with name: competencia1_modelito_lgbm


In [10]:
len(study.trials)

0

In [11]:
study.optimize(objective, n_trials=60)

Training until validation scores don't improve for 80 rounds


[I 2024-11-29 00:11:23,427] Trial 0 finished with value: 605808000.0 and parameters: {'num_leaves': 92, 'learning_rate': 0.16590205894469978, 'min_data_in_leaf': 1807, 'feature_fraction': 0.20108150681730297, 'bagging_fraction': 0.8148173312186163}. Best is trial 0 with value: 605808000.0.


Early stopping, best iteration is:
[53]	cv_agg's valid gan_eval: 1.21162e+08 + 2.07394e+06
Training until validation scores don't improve for 61 rounds


[I 2024-11-29 00:11:30,781] Trial 1 finished with value: 587139000.0 and parameters: {'num_leaves': 87, 'learning_rate': 0.43162019897259574, 'min_data_in_leaf': 207, 'feature_fraction': 0.9346582061887435, 'bagging_fraction': 0.8481750242583025}. Best is trial 0 with value: 605808000.0.


Early stopping, best iteration is:
[4]	cv_agg's valid gan_eval: 1.17428e+08 + 1.45146e+06
Training until validation scores don't improve for 78 rounds


...

Early stopping, best iteration is:
[26]	cv_agg's valid gan_eval: 1.21297e+08 + 1.99994e+06
Training until validation scores don't improve for 89 rounds


[I 2024-11-29 00:30:07,724] Trial 58 finished with value: 606242000.0 and parameters: {'num_leaves': 63, 'learning_rate': 0.12687282751588172, 'min_data_in_leaf': 398, 'feature_fraction': 0.3957237521780697, 'bagging_fraction': 0.7089981505932664}. Best is trial 43 with value: 610491000.0.


Early stopping, best iteration is:
[35]	cv_agg's valid gan_eval: 1.21248e+08 + 1.40004e+06
Training until validation scores don't improve for 63 rounds


[I 2024-11-29 00:30:15,653] Trial 59 finished with value: 601615000.0 and parameters: {'num_leaves': 30, 'learning_rate': 0.3598843862584943, 'min_data_in_leaf': 1050, 'feature_fraction': 0.4883058442811445, 'bagging_fraction': 0.9308002294087376}. Best is trial 43 with value: 610491000.0.


Early stopping, best iteration is:
[10]	cv_agg's valid gan_eval: 1.20323e+08 + 1.80388e+06


In [12]:
optuna.visualization.plot_optimization_history(study)

In [13]:
plot_param_importances(study)

In [14]:
plot_slice(study)

In [15]:
plot_contour(study)

In [16]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [17]:
study.best_trial.params

{'num_leaves': 16,
 'learning_rate': 0.06537511998403012,
 'min_data_in_leaf': 1295,
 'feature_fraction': 0.6760496941965302,
 'bagging_fraction': 0.6807642069212421}

In [18]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

139